# Web scraping av polisens händelser

En vanlig organisation som ingår i journalisternas nyhetsnät är polisen. På polisens hemsida finns en [lista över händelser i Sverige](https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/). Vi kan förvisso leta i denna händelselista manuellt efter något som intresserar oss, men varför inte göra det automatiskt med hjälp av Python?

Vi kan lösa det på två sätt:

1. **Skrapa RSS-flödet**. Vi kan skrapa polisens [RSS-flöde med händelser](https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?feed=rss). Detta är lättast eftersom [RSS](https://sv.wikipedia.org/wiki/RSS) är en standard för att utbyta information på webbplatser. Då kan vi också använda koden till alla webbplatser som har RSS-flöde. Men polisens RSS-flöde är begränsat, och alla händelser som finns på hemsidan finns inte med i RSS-flödet.
2. **Skrapa webbsidan**. Vi kan skrapa webbsidans HTML-kod. Då kan vi få ned allt som finns på hemsidan. Men om polisen ändrar sin hemsida så kan vårt program slutera fungera.

Vilken ska vi välja? Vi testar båda!

## Importera bibliotek

Importera de bibliotek som behövs för att skrapa hemsidor (`BeautifulSoup4` och `lxml`).

In [ ]:
# Installera bibliotek. Om det inte fungerar, testa att skriva pip3 i stället.
!pip install BeautifulSoup4
!pip install lxml

In [2]:
# Importera bibliotek.
from urllib import request
from lxml import html
from bs4 import BeautifulSoup
import ssl

def scrapewebpage(url):
    # Öppna URL och hämta HTML.
    UseSSL = True  # Om du får SSLError, ändra detta till False.
    if UseSSL:
        web = request.urlopen(url)
    else:
        web = request.urlopen(url, context=ssl._create_unverified_context())

    # Kontrollera att inga fel uppstod när URL öppnades.
    if web.getcode() == 200:
        return(web.read())
    else:
        print("Error %s reading %s" % str(web.getcode()), url)

# Hjälpfunktion som skrapar webbsidor och gör en soppa av allt ihop.
def makesoup(url):
    html = scrapewebpage(url)
    return(BeautifulSoup(html, "lxml-xml"))

## 1. Skrapa polisens händelser från RSS-flödet

RSS-flöde: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?feed=rss

In [3]:
# Läs in RSS-flödet och spara det i variablen rss_events.'
rss_events = makesoup("https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?feed=rss")

# Du kan också testa med ett annat RSS-flöde:
#rss_events = makesoup("https://www.svd.se/feed/articles.rss")

# Ta ut alla <item> från RSS-flödet.
rss_items = rss_events.find_all("item")

In [4]:
# Hur många händelser hittade vi?
print("Hittade " + str(len(rss_items)) + " händelser.")

Hittade 200 händelser.


In [5]:
# Visa senaste 10 händelserna. Plocka bort [0:10] för att visa alla.
for item in rss_items[0:10]:
    title = item.find("title").get_text()
    print(title)

2017-11-09 13:00, Övrigt, Västra Götaland
2017-11-09 12:47, Stöld/inbrott, Söderhamn
2017-11-09 12:25, Trafikolycka, Malmö
2017-11-09 11:52, Arbetsplatsolycka, Göteborg
2017-11-09 11:48, Trafikbrott, Luleå
2017-11-09 10:53, Trafikolycka, Kristianstad
2017-11-09 10:50, Trafikbrott, Sundsvall
2017-11-09 10:42, Rattfylleri, Piteå
2017-11-09 10:41, Stöld, Habo
2017-11-09 10:23, Brand, Stockholm


In [6]:
# En variabel som visar antalet sökresultat.
search_results = 0

# Visa bara trafikolyckor.
for item in rss_items:
    title = item.find("title").get_text()
    description = item.find("description").get_text()

    # Gör om texten till gemener, sök därefter efter trafikolyckor i Göteborg.
    if title.lower().find("trafikolycka") > -1 and title.lower().find("göteborg") > -1:
        search_results = search_results + 1
        print(title + ": " + description)
        print()

print("Hittade {0} resultat.".format(search_results))

2017-11-08 14:07, Trafikolycka, Göteborg: På Backavägen vid Leråkersmotet har två bilar kolliderat.

2017-11-08 07:58, Trafikolycka, personskada, Göteborg: På Tunnlandsgatan har en fotgängare blivit påkörd av en bilist.

2017-11-08 06:52, Trafikolycka, personskada, Göteborg: På Lillhagsvägen har en man kört omkull med sin moped.

2017-11-08 06:40, Trafikolycka, Göteborg: På Rävebergsvägen har två bilar kolliderat.

2017-11-07 20:40, Trafikolycka, Göteborg: I korsningen Skånegatan-Ullevigatan har två bilar kolliderat

Hittade 5 resultat.


In [7]:
# Gå igenom varje händelse och plocka ut datum, händelse och stad.
for item in rss_items:
    title = item.find("title").get_text()             # Titel på händelsen
    link = item.find("link").get_text()               # Länk till händelsen på polisens hemsida
    description = item.find("description").get_text() # Beskrivning av händelsen
    
    # Titlarna är skrivna i formatet "2017-11-07 20:40, Trafikolycka, Göteborg".
    # Då kan vi dela upp titeln i tre delar genom komma-tecknet
    # och på så vis plocka ut datum, händelse och ort.
    title_parts = title.split(",")                         # Dela sträng vid varje komma
    title_date = title_parts[0].strip()                    # Datum
    title_event = title_parts[1].strip()                   # Händelse
    title_city = title_parts[len(title_parts) - 1].strip() # Stad
    
    # Plocka ut händelser från Göteborg och Borås.
    if title_city in ["Göteborg", "Borås"]:
        print(title_date + " " + title_city + ", " + title_event + ": " + description)

2017-11-09 11:52 Göteborg, Arbetsplatsolycka: En man har skadats inne på en däckfirma.
2017-11-08 20:30 Göteborg, Brand: Lägenhetsbrand i Gårdsten
2017-11-08 14:07 Göteborg, Trafikolycka: På Backavägen vid Leråkersmotet har två bilar kolliderat.
2017-11-08 11:52 Borås, Brand: Lägenhetsbrand på Trandaredsgatan.
2017-11-08 10:33 Göteborg, Rån övrigt: I Lövgärdets centrum har två personer tilltvingat sig en blå lastbil från Postnord.
2017-11-08 07:58 Göteborg, Trafikolycka: På Tunnlandsgatan har en fotgängare blivit påkörd av en bilist.
2017-11-08 06:52 Göteborg, Trafikolycka: På Lillhagsvägen har en man kört omkull med sin moped.
2017-11-08 06:40 Göteborg, Trafikolycka: På Rävebergsvägen har två bilar kolliderat.
2017-11-07 20:40 Göteborg, Trafikolycka: I korsningen Skånegatan-Ullevigatan har två bilar kolliderat


## 2. Skrapa polisens händelser från webbsidans HTML-kod

Webbsida: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/ (notera att det finns händelser på flera sidor!)

In [8]:
# Läs in webbsidan och spara det i variablen html_events.
html_events = makesoup("https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/")

# Hur många sidor med händelser finns det?
html_pages = html_events.find_all("", "pageingnumbers")
pages = html_pages[0].get_text().strip().split(" ")
print("Hittade " + str(len(pages)) + " sidor med händelser.")

Hittade 6 sidor med händelser.


In [9]:
# Importera time för att kunna fördröja hämtningen.
import time

# Skapa en tom lista som vi ska spara händelserna i.
combined_events = []

# Gå igenom alla webbsidor.
for i in range(1, len(pages) + 1):
    # Visa meddelande om vad som händer och hur många sidor som har skrapats.
    print("Skrapar sida " + str(i) + ": https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=" + str(i))
    
    # Hämta HTML-kod från aktuell webbsida.
    page_soup = makesoup("https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=" + str(i))
    
    # Gå igenom alla händelser på sidan.
    for event in page_soup.find("ul", "page-list").find_all("li"):
        
        # Skapa ett dictionary-objekt av aktuell händelse.
        eventdict = { "title": event.find("h3").get_text().strip(),
                      "county": event.find("p").find("span", "news-date").get_text().strip(),
                      "description": event.find("p").get_text().strip(),
                      "link": "https://polisen.se" + event.find("h3").find("a").get("href")
                    }

        # Lägg till händelse i listan.
        combined_events.append(eventdict)
        
    # Lägg till 1.5 sekunders fördröjning för
    # för att inte överbelasta polisens server.
    time.sleep(1.5)

print("Klart.")

Skrapar sida 1: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=1
Skrapar sida 2: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=2
Skrapar sida 3: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=3
Skrapar sida 4: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=4
Skrapar sida 5: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=5
Skrapar sida 6: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=6
Klart.


In [10]:
# Hur många händelser hittade vi totalt?
print("Hittade " + str(len(combined_events)) + " händelser.")

Hittade 1111 händelser.


In [11]:
# Sök efter olyckor i Stockholm.
for event in combined_events:
    if event["title"].lower().find("olycka") > -1 and event["title"].lower().find("stockholm") > -1:
        print(event["title"])

2017-11-08 11:15, Trafikolycka, personskada, Stockholm
2017-11-06 17:53, Trafikolycka, smitning från, Stockholm
2017-11-06 17:00, Trafikolycka, Stockholm
2017-11-05 12:06, Trafikolycka, personskada, Stockholm
2017-11-03 17:10, Trafikolycka, personskada, Stockholm
2017-11-03 08:40, Arbetsplatsolycka, Stockholm


### Spara händelser till fil som kan öppnas i Excel

Spara datum, händelse och stad till en CSV-fil som kan öppnas i Excel.

In [12]:
# Skapa en funktion som sparar händelserna till en CSV-fil som kan öppnas i Excel.
# Filen är uppdelad i tre kolumner: datum, händelse och stad. Dessa separeras med komma-tecken.
def savefile(filename, date, event, city):
    text = date + "," + event + "," + city + "\n"
    f = open(filename, mode="a")   # En fil kan öppnas på många sätt (w=writing, a=append, r=reading)
    f.write(text)
    f.close()

In [13]:
print("Sparar...")
results = 0

# Gå igenom varje händelse, rad för rad, och bearbeta texten för att spara den till en textfil.
for event in combined_events:
    title = event["title"]               # Titel
    description = event["description"]   # Beskrivning av händelsen
    link = event["link"]                 # Länk till händelsen
    county = event["county"]             # Län
    
    # Titlarna är skrivna i formatet "2017-11-07 20:40, Trafikolycka, Göteborg".
    # Då kan vi dela upp titeln i tre delar genom komma-tecknet.
    title_parts = event["title"].split(",")   # Dela händelsen efter komma-tecken
    
    # Eftersom det ibland finns fler än tre komma-tecken kan vi behöva
    # hantera fel som uppstår genom try ... except.
    
    # Hantera fel för datum.
    try:
        date = title_parts[0].strip()
    except:
        date = ""
    
    # Hantera fel för händelser.
    try:
        event = title_parts[1].strip()
    except:
        event = ""
    
    # Hantera fel för stad.
    try:
        city = title_parts[len(title_parts) - 1].strip()
    except:
        city = ""
    
    # Sparar till filen "polisen.csv". Den hamnar i samma katalog som Jupyter Notebooks körs ifrån.
    results = results + 1
    savefile("polisen.csv", date, event, city)
    
print("Klart.")
print("Sparade {0} resultat.".format(results))

Sparar...
Klart.
Sparade 1111 resultat.
